# Laboratorio 4

Integrantes: 

    - Francis Aguilar - 22243 
    - Diego García - 22404 
    - Angela García -22869 

enlace al repositorio: https://github.com/angelargd8/lab4-modsim

# Ejercicio 1


## Teoría 
Defina y responda
1. Comparar la simulación de tiempo discreto (síncrona) y de tiempo continuo (asíncrona):

    a. Describir escenarios donde cada una es preferible

    b. Analizar las compensaciones computacionales (velocidad vs. precisión).

| tiempo discreto (síncrona) | tiempo continuo (asíncrona) |
|:-----|:--------:|
|Def:  El tiempo avanza en saltos fijos y en cada salto todos los eventos que ocurren en ese instante se procesan de forma sincronizada, antes de anvanzar al siguiente intervalo| Def: El tiempo fluye de manera continua y el motor de simulación salta directamente de un evento al siguiente sin procesar intervalos vacíos|
| Todos los eventos del intervalo se procesan juntos| Cada evento se procesa en el momento exacto|
| La precisión temporal está limitada por el tamaño de Δt| La presición temporal es muy alta, es exacta al instante de ocurrencia |
| Más simple de implementar | Es más compleja porque tiene una gestión de lista de eventos |
| En cuanto velocidad puede ser más lenta si Δt es pequeño, porque tiene muchos pasos| Es más eficiente si hay pocos eventos|
| En escenarios que es prefreribles es en simulaciones por turnos, modelos físicos simplificados y autómatas celulares | Este es mejor en escenariso de redes de colas, tráfico procesos industriales en tiempo real|
| Acerca de la compensacion computacional, es que es muy rápido de implementar y ejecutar cuando el intervalo de tiempo es grande, pero si se necesita mucha precisión temporal, se tiene que usar un intervalo de tiempo muy pequeño y eso incrementa exponencialmente el número de pasos y reduce la velocidad. La precisión está límitada por el tamaño de Δt, si es grande es menos preciso, pero más rápido. Y si Δt pequeño es más preciso, pero más lento| En cuanto a velocidad no simula intervalos vacíos, este solo salta de un evento a otro y eso puede ser muy eficiente si es que son pocos eventos. La desventaja que tiene es la gestión de la lista de eventos ordenada por tiempo, porque esto si añade sobrecarga ocmputacional, si hay muchos eventos por segundo. Aunque, su presición es muy alta, ya que maneja tiempos exactos para cada evento y no depende de Δt, sino que de la exactitud de los cálculos y del reloj del simulador|

2. Sobre los mecanismos de gestión de eventos:

    a. Explicar cómo las colas de eventos gestionan los cambios de estado.

    Las colas de eventos gestionan los cambios de estados, ya que estos son una estructura de datos que tienen todos los elementos futuros programados en la simulación, entonces cada evento tiene tiempo de ocurrencia, tiempo de evento y datos asociados. La cola siempre está ordenada por tiempo de ocurrencia para que el próximo evento se accesible de una manera rápida. Con el ciclo típico de simulacion con cola de eventos primero se inicializa, creando los primeros eventos con las condiciones iniciales y se insertan en la cola con sus tiempos de ejecución. Luego en el bucle de simulación mientras existan eventos y no se llegue al tiempo final, se extrae el evento más próximo, se avanza elreloj de simulación al tiempo de ese evento, se procesa el evento para actualizar estados de entidades y se genera nuevos eventos que surjan como consecuencia y se añaden a la cola, por último, se detiene cuando no hay más eventos o se alcanza la condición de fin. 

    b. Analizar la gestión de prioridades para eventos concurrentes.

    La gestión de prioridades para eventos concurrentes es algo que puede llegar a ser delicado al trabajar con colas de eventos, se suele trabajar con tiempo continuo. La gestión de prioridades lo que define es en qué orden se procesan para mantener la coherencia y reproducibilidad del modelo. Se usa como criterio principal que se ordenan por el tiempo de ocurrencia. Y como criterios secundarios que se puede usar una prioridad fija por tipo de evento o reglas dependientes del estado del sistema. Y para los resultados si tiempo y tipo son iguales, se usa el orden FIFO de la cola para asegurar resultados deterministas. Entonces, si se tiene una buena gestión se evitan errores como resultados no reproducibles, estados invalidos, inconsistencias en la lógica y se implementan estructuras como priority queues que perminten ordenar por múltiples claves y mantener un orden.


In [ ]:
# %pip install simpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import random
import math
from collections import defaultdict, Counter
import simpy
from dataclasses import dataclass
from enum import Enum, auto

# Ejercicio práctico: marco de simulación

# escenario: propagación de enfermedades en un hospital

# arquitecturas: 
# 1. tiempo discreto - actualizaciones diarias
# 2. tiempo continuo - infecciones impultadas por eventos 
# atributos necesarios de cada agente
#tipos de eventos a monotorizar
#metricas de salida esperadas

#estados 
class Salud(Enum):
    S = auto() #suceptible
    E= auto() #expuesto
    I= auto() #infeccioso
    R= auto() #recuperado

#modelo
@dataclass #los dataclass los siento muy c xd
class Agent: 
    id: int
    rol: str
    estado: Salud= Salud.S
    t_inf_ini: float = None
    t_rec: float = None

@dataclass
class Params:
    T: int = 30 #dias a simular
    tasa_contacto: float = 2.0 #contactos promedio por dia por agente
    contactos_dia: int = 40 #contactos aleatorios por dia 
    beta_contacto: float = 0.15 #riesgo base por contacto
    inc_media: float = 3.0 #media de incubacion
    inf_media: float = 5.0 #media de infeccioso

def utilidad(mean):
    return random.expovariate(1.0/mean)

def p_trans(beta):
    prob_por_contacto = 1.0 - math.exp(-beta)
    return prob_por_contacto

def tiempoDiscreto(agents, params):
    semilla = 42
    random.seed(semilla)
    env = simpy.Environment()
    #metricas

    incidencia = {d: 0 for d in range(params.T)}
    prevalencia = [] #T, i
    

    #flujo - por dia en d tiempo
    def tick_diario(env): 
        for d in range(params.T):    
        #generar contacots dia
            ids = list(range(len(agents)))
            #por cada contacto en contactos
            for i in range(params.contactos_dia):
                a, b= random.sample(ids, 2)
                A, B= agents[a], agents[b]

                #intentar transmision
                for emisor, receptor in [(A, B), (B, A)]:
                    if emisor.estado ==Salud.I and receptor.estado == Salud.S:
                        if random.random() < p_trans(params.beta_contacto):
                            receptor.estado = Salud.E
                            receptor.t_inf_ini = d + utilidad(params.inc_media)
                            receptor.t_rec = receptor.t_inf_ini + utilidad(params.inf_media)
                            incidencia[d] += 1
                            # prevalencia.append((d, receptor))


            # propagacion por marcas
            for a in agents: 
                if a.estado == Salud.E and a.t_inf_ini is not None and a.t_inf_ini <= d+1:
                    a.estado = Salud.I
                if a.estado == Salud.I and a.t_rec is not None and a.t_rec <= d+1:
                    a.estado = Salud.R

            # recolectar metricas
            I = sum(a.estado == Salud.I for a in agents)
            prevalencia.append((d+1, I))

            #avanzar al siguiente dia
            yield env.timeout(1)

    env.process(tick_diario(env))
    env.run(until=params.T)
    return {"incidencia": incidencia, "prevalencia": prevalencia}

def tiempoContinuo(agents, params: Params):
    random.seed(42)
    env = simpy.Environment()

    incidencia_t = [] # tiempos exactos de infección 
    incidencia_dia = defaultdict(int) # conteo por día 
    prevalencia_t = []  #snapshots diarios

    def registrar_infeccion(t: float):
        incidencia_t.append(t)
        incidencia_dia[int(t)] += 1

    def progression(env, a: Agent):
        # Incubacion
        yield env.timeout(utilidad(params.inc_media))
        if a.estado != Salud.E:
            return
        a.estado = Salud.I
        # Infeccioso
        yield env.timeout(utilidad(params.inf_media))
        if a.estado == Salud.I:
            a.estado = Salud.R

    def recovery_if_I(env, a: Agent):
        # Para quien ya está en I al inicio: programa su recuperación una sola vez
        yield env.timeout(utilidad(params.inf_media))
        if a.estado == Salud.I:
            a.estado = Salud.R

    def contactos_actor(env, a: Agent):
        # Proceso generador de contactos para el agente
        while env.now < params.T:
            wait = utilidad(1.0 / params.tasa_contacto) if params.tasa_contacto > 0 else params.T
            yield env.timeout(wait)

            #elegir otro
            otros = [x for x in agents if x.id != a.id]
            if not otros:
                continue
            b = random.choice(otros)

            # Intentar transmision
            for em, rc in ((a, b), (b, a)):

                if em.estado == Salud.I and rc.estado == Salud.S:
                    if random.random() < p_trans(params.beta_contacto):
                        rc.estado = Salud.E
                        registrar_infeccion(env.now)
                        env.process(progression(env, rc))

    # sino hay infecciosos iniciales siembra uno
    hay_I = any(a.estado == Salud.I for a in agents)
    if not hay_I and agents:
        agents[0].estado = Salud.I

    #recuperación
    for a in agents:
        if a.estado == Salud.I:
            env.process(recovery_if_I(env, a))

    # procesos de contacto
    for a in agents:
        env.process(contactos_actor(env, a))

    # snapshot diario 
    def snapshot_diaria(env):
        while env.now < params.T:
            I = sum(x.estado == Salud.I for x in agents)
            prevalencia_t.append((int(env.now), I))
            yield env.timeout(1)
    env.process(snapshot_diaria(env))

    env.run(until=params.T)

    return { "incidencia_t": incidencia_t, "incidencia_dia": dict(incidencia_dia), "prevalencia_t": prevalencia_t }

In [3]:
from copy import deepcopy 

# hospital
agents_A = [Agent(i, "paciente" if i < 15 else "personal") for i in range(25)]
agents_B = [Agent(i, "paciente" if i < 15 else "personal") for i in range(25)]

# Discreto
P = Params()
#sembrar un caso inicial
agents_A[0].estado = Salud.I
agents_A[0].t_rec = utilidad(Params().inf_media)
res_discreto = tiempoDiscreto(deepcopy(agents_A), P)
print("-- discreto -- ")
print("Incidencia por día :\n ", res_discreto["incidencia"])
print("Prevalencia : \n" , res_discreto["prevalencia"])

#b
agents_B[0].estado = Salud.I
agents_B[1].estado = Salud.I
res_cont = tiempoContinuo(deepcopy(agents_B), P)
print("\n-- continuo --")
print("Infecciones (tiempos):", res_cont["incidencia_t"])
print("Incidencia por día:", res_cont["incidencia_dia"])
print("Prevalencia ", res_cont["prevalencia_t"])


-- discreto -- 
Incidencia por día :
  {0: 1, 1: 0, 2: 3, 3: 2, 4: 2, 5: 1, 6: 0, 7: 1, 8: 1, 9: 0, 10: 1, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0, 16: 1, 17: 0, 18: 1, 19: 0, 20: 1, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0}
Prevalencia : 
 [(1, 2), (2, 2), (3, 3), (4, 4), (5, 5), (6, 4), (7, 4), (8, 5), (9, 3), (10, 3), (11, 2), (12, 2), (13, 3), (14, 2), (15, 3), (16, 4), (17, 4), (18, 5), (19, 4), (20, 5), (21, 4), (22, 3), (23, 2), (24, 1), (25, 0), (26, 0), (27, 0), (28, 0), (29, 0), (30, 0)]

-- continuo --
Infecciones (tiempos): [1.1047400053375287, 3.1818517394059684, 10.610633456160713, 12.508767960303466, 12.898323451553098, 12.978188575070948, 13.10305866859496, 13.879364980524894, 14.192871421100953, 14.595768824811277, 15.040153183382227, 15.426755937204417, 15.698902209734, 16.650449333637184, 17.28867572251684, 17.46200267269659, 18.060545417415867, 23.26930019276963, 23.491346975554617, 26.845619063032427]
Incidencia por día: {1: 1, 3: 1, 10: 1, 12: 3, 13

# Ejercicio 2
### Teoría 

1. Considere la taxonomía de rasgos, clasifique y justifique cada uno de los siguientes (discreto, contínuo o
relacional) 

| Aspecto | Clasificación | Justificación |
|:-----|:--------|:-----|
|Edad|Contnuo |Se mide en una escala numérica y puede tomar cualquier valor dentro de un rango. Es un atributo cuantitativo que refleja magnitud y permite operaciones matemáticas.|
|Profesión|Discreto |Corresponde a categorías nominales (médico, ingeniero, profesor, etc.), mutuamente excluyentes y no ordenadas.|
|Redes de amistad| Relacional |Describe vínculos o relaciones entre individuos y no es un atributo de un solo individuo, sino de pares o grupos de individuos.|
|Estado de vacunación|Discreto|Se expresa en categorías finitas. No se mide en escala numérica continua, sino en valores categóricos.

2. Calcule el tamaño total del espacio de parámetros y responda ¿cómo podría afectar esto al tiempo de
ejecución de la simulación? Para esto, considere un escenario de 10,000 agentes con:

a. 3 rasgos continuos (p. ej., tasa de movilidad).  
b. 2 rasgos discretos (p. ej., ocupación)

### Ejercicio práctico

Construya un perfil poblacional heterogéneo para un escenario de “Respuesta a una pandemia urbana”, para ello
considere las siguientes tareas

In [1]:
import random

class Persona:
    def __init__(self, id_persona, hogar_id):
        # Identificadores
        self.id_persona = id_persona
        self.hogar_id = hogar_id  # rasgo relacional

        # Rasgos discretos
        self.ocupacion = random.choice(["trabajador_esencial", "teletrabajador", "desempleado", "estudiante"])
        self.estado_salud = random.choice(["sano", "infectado", "recuperado", "vacunado"])
        self.nivel_educativo = random.choice(["primaria", "secundaria", "universitario", "posgrado"])

        # Rasgos continuos
        self.tasa_movilidad = round(random.uniform(0.0, 1.0), 2)  # 0 = no se mueve, 1 = muy móvil
        self.nivel_interaccion = round(random.uniform(0, 50), 1)  # contactos diarios promedio

    def __repr__(self):
        return (f"Persona(id={self.id_persona}, hogar={self.hogar_id}, "
                f"ocupacion={self.ocupacion}, salud={self.estado_salud}, "
                f"educacion={self.nivel_educativo}, movilidad={self.tasa_movilidad}, "
                f"interaccion={self.nivel_interaccion})")

# Ejemplo: generar una pequeña población
poblacion = [Persona(id_persona=i, hogar_id=random.randint(1, 10)) for i in range(5)]

for persona in poblacion:
    print(persona)


Persona(id=0, hogar=8, ocupacion=estudiante, salud=recuperado, educacion=primaria, movilidad=0.93, interaccion=21.2)
Persona(id=1, hogar=9, ocupacion=teletrabajador, salud=infectado, educacion=universitario, movilidad=0.35, interaccion=16.2)
Persona(id=2, hogar=8, ocupacion=estudiante, salud=recuperado, educacion=posgrado, movilidad=0.24, interaccion=6.3)
Persona(id=3, hogar=4, ocupacion=estudiante, salud=sano, educacion=posgrado, movilidad=0.89, interaccion=2.2)
Persona(id=4, hogar=8, ocupacion=teletrabajador, salud=vacunado, educacion=secundaria, movilidad=0.32, interaccion=18.1)


 ¿Cómo encodearías los trabajadores de la salud vs los maestros para evitar sesgo?

Si los datos muestran que el 20% de las enfermeras trabajan en turnos de noche, ¿cómo
validarías esto en tu modelo?

# Ejercicio 4
## Teoria

4. Dadas las curvas resultante de dos simulaciones que se muestran abajo, considere que cada una de estas 

a. Resultado A: Muestra eventos de superpropagación claros (modelado discreto de tiempo)  
i. Los picos se dan precisamente en intervalos semanales  
ii. No hay infecciones reportadas entre los días 6-7 o 13-14  
iii. Distribución de rasgos del agente:  
1. Edad – Discreta - [0-17, 18-65, 65+]  
2. Movilidad – Continua - 0.1–5.0 (Contactos diarios)  


b. Resultado B: Muestra patrones de transmision uniformes (Modelo de tiempo continuo)  
i. Los picos se producen a intervalos irregulares (días 3.2, 8.7 y 14.1).  
ii. Transmisión de bajo nivel entre brotes importantes.  
iii. Distribución de rasgos del agente  
1. Edad – Continua – 0-100 años  
2. Inmunidad – Continua – 0.0-1.0 score de protección  


#### c. Reponda:   

i. Para el resultado A:   
1. ¿Por qué los intervalos perfectos de 7 días sugieren un modelado de tiempo discreto?  

Esto se debe a que los contagios aparecen acumulados en un único punto del tiempo, generando picos muy altos en días específicos y ausencia de casos significativos en los días intermedios.

2. ¿Cómo podría esto distorsionar la dinámica de transmisión en el mundo real?   

En el mundo real los contagios ocurren de manera continua y no concentrados en intervalos rígidos. Al acumular los casos en ciertos días, el modelo termina exagerando la magnitud de los picos y subestimando la transmisión en los periodos entre ellos. Esto hace que se genera una visión periódica de la epidemia, que no refleja adecuadamente las variaciones diarias o la influencia de factores sociales. 

ii. Para el resultado B:    
1. ¿Qué evidencia indica un procesamiento continuo?    

En el resultado B se observa que el número de infecciones varía de manera fluida a lo largo del tiempo, sin saltos bruscos concentrados en intervalos fijos. La curva muestra oscilaciones suaves y continuas, con picos y descensos graduales, lo cual es evidencia de un procesamiento en tiempo continuo. En este tipo de modelado, los contagios se actualizan de forma constante

2. ¿Por qué las infecciones entre picos son visibles aquí, pero no en el Resultado A? 

En el modelo B las infecciones se ven también entre los picos porque no están limitadas a intervalos fijos. A diferencia del modelo A, donde los contagios se acumulan y aparecen de golpe en un solo día, aquí los casos se reparten de forma continua a lo largo del tiempo. Esto permite mostrar que siempre hay cierto nivel de transmisión, incluso fuera de los picos, dando una visión más realista de cómo la epidemia se mantiene activa día a día.


iii. El resultado A muestra picos de tamaño idéntico a pesar de las diferentes puntuaciones de movilidad. ¿Es esto realista? ¿Por qué?  

No es realista, en la vida real, la movilidad de la población influye directamente en la probabilidad de contacto entre personas y, por lo tanto, en la transmisión de la enfermedad. Si la movilidad aumenta, lo esperable es que los contagios también se incrementen, generando picos más altos; mientras que si la movilidad disminuye, los picos deberían ser más bajos.


iv. El resultado B muestra algunos eventos superpropagadores (el pico del día 8.7 es tres veces mayor que otros). ¿Qué rasgo(s) podría(n) explicar esto?  

El resultado B muestra un evento de superpropagación alrededor del día 8.7, donde el pico es mucho más alto que los demás. Esto puede pasar porque algunas personas tienen muchos más contactos que otras, porque ocurren eventos con aglomeraciones, porque ciertos individuos son más contagiosos, o porque las condiciones del lugar (cerrado, sin ventilación, sin medidas de prevención) facilitan el contagio. Por estas razones, en lugar de un crecimiento uniforme, aparecen picos muy grandes en ciertos momentos.


v. Pronga:  
1. Una prueba de sensibilidad temporal para el resultado A (p. ej., repetir la ejecución con intervalos de 12 horas frente a 24 horas).

Para el resultado A, una forma de probar su sensibilidad temporal sería repetir la simulación usando un intervalo de 12 horas en lugar de 24. Con esto se puede observar si los picos semanales son un artefacto del paso de tiempo que se eligió. Si al reducir el intervalo los picos se suavizan y aparecen más contagios entre ellos, significaría que el modelo discreto está exagerando la periodicidad.

2. Una prueba de aleatorización de rasgos para el resultado B para aislar los efectos de inmunidad frente a la edad

En el resultado B, se puede hacer una prueba de aleatorización para separar los efectos de la inmunidad y la edad. Esto consiste en mezclar los valores de inmunidad manteniendo fija la edad, y luego al revés: mezclar las edades manteniendo fija la inmunidad. Así se puede ver qué factor tiene más peso en la propagación. Si al cambiar la inmunidad los resultados varían mucho, la inmunidad es la clave; si cambian más al alterar la edad, entonces la edad es la que explica mejor los patrones de transmisión.



## Referencias:
- Reyes, C. (2018, marzo 24). Relación entre sistemas de control en tiempo continuo y en tiempo discreto. Control digital. Herramientas de cálculo. https://herramientasdecalculo.com/2018/03/24/1-2-relacion-entre-sistemas-de-control-en-tiempo-continuo-y-en-tiempo-discreto-control-digital/

- Juanweb. (2023, octubre 1). Simulación de eventos discretos con SimPy en Python. CodigosPython. https://codigospython.com/simulacion-de-eventos-discretos-con-simpy-en-python/

 